In [ ]:
! pip install mip

In [ ]:
# Observação: branch-and-bound ineficiente (gera cópias desnecessárias do modelo).
# Utilizado para fins didáticos na disciplina Otimização Linear e Inteira.

import sys
from mip import *

EPS = 1e-6

####################
# Dados de entrada #
####################

# lendo dados do arquivo csv
items = []
csv = open("mochila.csv", "r")
lines = csv.readlines()
for line in lines[1:]:
    data = line.split(";")
    item = {}
    item["name"] = int(data[0])
    item["weight"] = float(data[1])
    item["value"] = float(data[2])
    items.append(item)
csv.close()

In [ ]:
# índices dos itens, para simplificar loops
indices = range(len(items)) # indices = [0,1,...,len(itens)-1]

# lendo capacidade da linha de comando
capacity = 6.0

In [ ]:
#########################
# Modelagem do problema #
#########################

# criando modelo (que terá restrições de integralidade relaxadas)
knapsack = Model("Knapsack", solver_name=CBC, sense=MAXIMIZE)

# criando variáveis do modelo
x = [ knapsack.add_var(ub=1, name="x_%d" % i) for i in indices ]

# criando função objetivo
knapsack += xsum( x[i] * items[i]["value"] for i in indices )

# criando restrição de capacidade
knapsack += xsum( x[i] * items[i]["weight"] for i in indices ) <= capacity, "capacity"

In [ ]:
def add_cut(p, x, items, capacity):
    """
    Adiciona restrições de cover violadas
    """


In [ ]:
####################
# Branch-and-bound #
####################

knapsack.verbose = 0

# inicializando pilha de problemas a resolver (nós da árvore a avaliar)
L = [ knapsack ]
z = 0 # custo da melhor solução encontrada (zero inicialmente)

# variável que indica o número de nós avaliados
nro_nos = 0

while L:
    # descomente as linha abaixo para pausar a execução
    print("Pressione ENTER para continuar")
    input()

    # imprimindo número do nó que será resolvido
    nro_nos += 1
    print("\n-----------------------")
    print(" Resolvendo nó %d" % nro_nos)    
    print("-----------------------\n")

    # seleção de subproblema
    p = L.pop()
    x = {(i['name']-1) : p.var_by_name('x_%d' % (i['name']-1)) for i in items}
    
    # adicionando cortes 
    n_cuts = 0
    has_cut = True
    while has_cut:   
        p.optimize()
        print(p.status)
        if p.status != OptimizationStatus.OPTIMAL:
            print('Not optimal')
            break
        print('\nNó {nro_nos} tem custo {p.objective_value:.5f}'.format(**locals()))
        selected = [i for i in indices if x[i].x >= 0 + EPS]
        values = [x[i].x for i in selected]
        print("selected items: {}".format(selected))
        print("selected values: {}".format(values))
        
        has_cut = 0 #add_cut(p, x, items, capacity)
        n_cuts += has_cut
    print('Total de cortes inseridos: {n_cuts}\n'.format(**locals()))

    # tentando podar o nó atual (por inviabilidade ou usando limites)
    if p.status != OptimizationStatus.OPTIMAL:
        print("\n-----> Poda por inviabilidade!\n")
        continue
    if p.objective_value <= z: # alterar para z + 1 (solução inteira!)
        print("\n-----> Poda por limite: %f <= %f!\n" % (p.objective_value, z))
        continue
    

    # teste de integralidade: procurando variável com valor fracionário
    var_frac = None
    for x in p.vars:
        if x.x > 0 + EPS and x.x < 1 - EPS:
            var_frac = x.name
            break
    if not var_frac:
        print("\n-----> Solução inteira encontrada de custo %f!\n" % p.objective_value)
        if p.objective_value > z:
            z = p.objective_value
        continue

    print("\n-----> Criando ramificações ({var_frac} <= 0) e ({var_frac} >= 1)\n".format(**locals()))
    
    # criando ramo "var <= 0"
    p0 = p.copy() # (note que criar uma cópia do modelo é uma opção bastante custosa)
    p0.var_by_name(var_frac).ub = 0
    
    # criando ramo "var >= 1"
    p1 = p # (não é necessário criar duas cópias, já que *p* não será mais utilizado)
    p1.var_by_name(var_frac).lb = 1

    # adicionando os novos nós (ramos) na pilha L
    L.append(p0)
    L.append(p1)

In [ ]:
# imprimindo o resultado (custo da melhor solução)
print("\n##################################")
print("Total de nós avaliados: {nro_nos}".format(**locals()))
print("Custo da solução ótima: {z}".format(**locals()))
print("##################################\n")